In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

data = [
    {'title': 'Iron Man', 'genre': 'Action Sci-Fi Hero'},
    {'title': 'Titanic', 'genre': 'Romance Drama'},
    {'title': 'Avengers', 'genre': 'Action Hero Team'},
    {'title': 'The Notebook', 'genre': 'Romance Drama'},
    {'title': 'Thor', 'genre': 'Action Hero God'},
]
df = pd.DataFrame(data)

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['genre'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def recommend(title):
    idx = df[df['title'] == title].index[0]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    top_indices = [i[0] for i in sim_scores[1:3]]
    return df['title'].iloc[top_indices]

print(f"คนที่ดู 'Iron Man' น่าจะชอบ: \n{recommend('Iron Man')}")

คนที่ดู 'Iron Man' น่าจะชอบ: 
2    Avengers
4        Thor
Name: title, dtype: object


In [5]:
from sklearn.decomposition import TruncatedSVD
import numpy as np
ratings = pd.DataFrame([
    [5, 5, 0, 0], # User 1 (สายบู๊)
    [5, 4, 0, 0], # User 2 (สายบู๊)
    [0, 0, 5, 4], # User 3 (สายรัก)
    [0, 0, 4, 5], # User 4 (สายรัก)
], columns=['Iron Man', 'Avengers', 'Titanic', 'Notebook'])

print("ตารางคะแนนเดิม:\n", ratings)

# (Matrix Factorization)
# "Latent Features"
SVD = TruncatedSVD(n_components=2) # บีบเหลือ 2 มิติ (เช่น แกนบู๊ กับ แกนรัก)
decomposed_matrix = SVD.fit_transform(ratings.T) # Transpose เพื่อหาความสัมพันธ์ระหว่างหนัง

correlation_matrix = np.corrcoef(decomposed_matrix)

#  ถ้าชอบ Iron Man (Index 0) จะแนะนำอะไร?
iron_man_corr = correlation_matrix[0]
def recommend_robust(movie_name):
    movie_idx = list(ratings.columns).index(movie_name)
    correlation_product = correlation_matrix[movie_idx]
    
    sorted_indices = correlation_product.argsort()[::-1]
    
    print(f"🔎 กำลังหาหนังที่เหมือน: {movie_name}")
    print("-" * 30)
    
    count = 0
    for idx in sorted_indices:
        if idx != movie_idx: # ข้ามตัวเอง (Iron Man)
            score = correlation_product[idx]
            recommended_movie = ratings.columns[idx]
            print(f"แนะนำ: {recommended_movie} (ความเหมือน {score*100:.2f}%)")
            
            count += 1
            if count >= 2: # แนะนำแค่ 2 เรื่องพอ
                break
recommend_robust('Iron Man')

ตารางคะแนนเดิม:
    Iron Man  Avengers  Titanic  Notebook
0         5         5        0         0
1         5         4        0         0
2         0         0        5         4
3         0         0        4         5
🔎 กำลังหาหนังที่เหมือน: Iron Man
------------------------------
แนะนำ: Avengers (ความเหมือน 100.00%)
แนะนำ: Titanic (ความเหมือน -100.00%)


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten, Dense
from tensorflow.keras.models import Model

n_users = 100
n_items = 50

user_input = Input(shape=(1,), name='User_Input')
item_input = Input(shape=(1,), name='Item_Input')

user_embedding = Embedding(n_users, 8, name='User_Embed')(user_input)
item_embedding = Embedding(n_items, 8, name='Item_Embed')(item_input)

dot_product = Dot(axes=2, name='Dot_Product')([user_embedding, item_embedding])
output = Flatten()(dot_product) # แปลงเป็นค่าคะแนนเดียว

model = Model([user_input, item_input], output)
model.compile(optimizer='adam', loss='mse') # MSE เพราะทำนายคะแนน 1-5

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ User_Input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Item_Input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ User_Embed (Embedding)        │ (None, 1, 8)              │             800 │ User_Input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Item_Embed (Embedding)        │ (None, 1, 8)              │             400 │ Item_Input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Dot_Product (Dot)             │ (None, 1, 1)              │               0 │ User_Embed[0][0],          │
│                               │                           │                 │ Item_Embed[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 1)                 │               0 │ Dot_Product[0][0]          │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,200 (4.69 KB)

 Trainable params: 1,200 (4.69 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
import numpy as np
from sklearn.linear_model import LogisticRegression

X_train = np.array([
    [25, 1, 1, 1], # หนุ่ม สนใจกีฬา ดูกีฬาบนมือถือ -> น่าจะคลิก
    [45, 0, 1, 0], # อายุเยอะ ไม่สนกีฬา ดูกีฬาบนคอม -> ไม่น่าคลิก
    [30, 1, 0, 0], # กลางคน ดูแฟชั่นบนมือถือ -> อาจจะคลิก
    [22, 1, 1, 0], # เด็ก ดูกีฬาบนมือถือ แต่ไม่ได้สนใจพิเศษ -> 50/50
    [50, 0, 0, 1], # อายุเยอะ ดูแฟชั่น แต่สนใจกีฬา -> ไม่น่าคลิก
])
# ผลลัพธ์จริง (1=คลิก, 0=ไม่คลิก)
y_train = np.array([1, 0, 1, 0, 0])

model = LogisticRegression()
model.fit(X_train, y_train)

# User คนนี้: อายุ 28, ใช้มือถือ, สนใจกีฬา
Current_User_Features = [28, 1, 1] # 3 feature แรก

# Ad ที่มีให้เลือก 2 ตัว
Ad_A_Sport = [1] # เป็น Ad กีฬา
Ad_B_Fashion = [0] # เป็น Ad แฟชั่น

# สร้าง Feature Input สำหรับการทำนาย
# เอา User feature + Ad feature มารวมกัน
Input_A = [Current_User_Features + Ad_A_Sport] # [28, 1, 1, 1]
Input_B = [Current_User_Features + Ad_B_Fashion] # [28, 1, 1, 0]

#  ทำนายความน่าจะเป็น (pCTR - Predicted Click-Through Rate)
# เราสนใจค่าที่ index [1] คือโอกาสที่จะเป็น "1" (คลิก)
prob_A = model.predict_proba(Input_A)[0][1]
prob_B = model.predict_proba(Input_B)[0][1]

print(f"โอกาสคลิก Ad A (กีฬา): {prob_A*100:.2f}%")
print(f"โอกาสคลิก Ad B (แฟชั่น): {prob_B*100:.2f}%")

# ระบบจะเลือก Ad ที่คะแนน pCTR * ราคาประมูล สูงที่สุดไปแสดง
if prob_A > prob_B:
    print("✅ ตัดสินใจโชว์: Ad A (กีฬา)")
else:
    print("✅ ตัดสินใจโชว์: Ad B (แฟชั่น)")

โอกาสคลิก Ad A (กีฬา): 54.83%
โอกาสคลิก Ad B (แฟชั่น): 48.19%
✅ ตัดสินใจโชว์: Ad A (กีฬา)


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

N_USERS = 10000  # มี User 1 หมื่นคน
N_ITEMS = 5000   # มีหนัง 5 พันเรื่อง
EMBED_DIM = 32   # ขนาด Vector ที่จะเรียนรู้ (ยิ่งเยอะยิ่งฉลาด แต่เทรนนาน)

user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')

user_embedding = Embedding(input_dim=N_USERS, output_dim=EMBED_DIM, name='user_embed')(user_input)
item_embedding = Embedding(input_dim=N_ITEMS, output_dim=EMBED_DIM, name='item_embed')(item_input)
user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)

concat = Concatenate()([user_vec, item_vec])

x = Dense(128, activation='relu')(concat)
x = Dropout(0.2)(x) # ป้องกัน Overfitting
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)

output = Dense(1, activation='linear', name='prediction')(x)
model = Model(inputs=[user_input, item_input], outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

model.summary()

user_id_to_predict = np.array([100])
movie_id_to_predict = np.array([50])
predicted_rating = model.predict([user_id_to_predict, movie_id_to_predict])
print(f"โมเดลทายว่า User 100 จะให้คะแนนหนังเรื่องนี้: {predicted_rating[0][0]:.2f} คะแนน")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ item_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ user_embed (Embedding)        │ (None, 1, 32)             │         320,000 │ user_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ item_embed (Embedding)        │ (None, 1, 32)             │         160,000 │ item_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 32)                │               0 │ user_embed[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_2 (Flatten)           │ (None, 32)                │               0 │ item_embed[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 64)                │               0 │ flatten_1[0][0],           │
│                               │                           │                 │ flatten_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │           8,320 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │           8,256 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 64)                │               0 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ prediction (Dense)            │ (None, 1)                 │              65 │ dropout_1[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 496,641 (1.89 MB)

 Trainable params: 496,641 (1.89 MB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
โมเดลทายว่า User 100 จะให้คะแนนหนังเรื่องนี้: 0.01 คะแนน
